In [12]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
spark = SparkSession.builder.master("local[4]").appName("giveatry").getOrCreate()
df = spark.read.format('csv').options(delimiter=',',header="True",inferschema="True").load("C:\\Users\\brhng\\Desktop\\Advertising.csv")

In [ ]:
#Data Bender Part
df.describe().show() #->describe
df.columns() #->columns names list
df.select(["column1","column2"]).show() #-> columns selection
df.printSchema() #-> df.info()
df.dtypes #-> columns types
df.withColumn("column3",df["column2"]+125) #-> inplace=false 
df.drop("column1")#-> inplace=false
df.withColumnRenamed("Name","New Name") #-> column renamed
df.na.drop() #-> dropping nan values (row)
    #how == any(default at least 1) or all(all of them should be nan for dropping) 
    #threshold = 2(at leats 2 non value)
    #subset["column_name"] (just column_name daki nanlar silinir)
df.na.fill("0",["column1","column2"]) #-> 0 ile dolduruyor
df.filter("column1 <= 1000").select(["column1","column2"]).show()
df.filter(~(df["column1"]<2000)&(df["column1"]!=0)).show()
df.groupBy("columne1"). sum(),mean(),count(),max(),...
df.agg({"column1":"sum"}).show()
df.dropDuplicates(["department", "salary"])  # ikisnin de aynı olanlar duplicate sayılıcak
distinct_amount = str(df.distinct()) #unique

In [ ]:
#pivot sabit demek basketballdaki gibi bir sutünü sabit tutup diyerlerini
pivotDf = df.groupby("Product").pivot("Country").sum("Amount")
pivotDf.printSchema()
pivotDf.show(truncate = False)
#her productın ülkeler özelindeki satış amountları tablosu

# +-------+------+-----+------+----+
# |Product|Canada|China|Mexico|USA |
# +-------+------+-----+------+----+
# |Orange |null  |4000 |null  |4000|
# |Beans  |null  |1500 |2000  |1600|
# |Banana |2000  |4000 |null  |1000|
# |Carrots|2000  |1200 |null  |1500|
# +-------+------+-----+------+----+

In [ ]:
#word count example
veri_seti ="C:\\Users\\brhng\\Desktop\\ibbdata-lojistic.txt"
istac_rdd =sc.textFile(veri_seti)
istac_rdd.count() #satır sayısı
istac_rdd.take(5) #ilk 5 satır
istac_rdd_kelimeler = istac_rdd.flatMap(lambda satir: satir.split(" ")) #satırlardan kelimelere
istac_rdd_kelimeler_sayilari = istac_rdd_kelimeler.map(lambda kelime:(kelime,1)) #wordcount
istac_rdd_kelimeler_sayilar_reduce = istac_rdd_kelimeler_sayilari.reduceByKey(lambda x,y:(x+y))
istac_rdd_kelimeler_sayilar_reduce_sort = istac_rdd_kelimeler_sayilar_reduce.map(lambda x:(x[1],x[0]))
istac_rdd_kelimeler_sayilari_reduce_sort.sortByKey(False).take(20)

In [ ]:
#Creatae DataFrame
data = [("James", "", "William", "36636", "M", 3000), ("Michael", "Smith", "", "40288", "M", 4000), ("Robert", "", "Dawson", "42114", "M", 4000), 
        ("Maria", "", "Jones", "39192", "F", 4000)]

schema = StructType([
    StructField("firstname", StringType(), True),\
    StructField("middlename", StringType(), True),\
    StructField("lastname", StringType(), True),\
    StructField("id", StringType(), True),\
    StructField("gender", StringType(), True),\
    StructField("salary", IntegerType(), True)\
    ])

df = spark.createDataFrame(data = data, schema = schema)

In [17]:
#ML with pyspark
#-> featurlaresimi tek bir feature olarak birleştiriyoruz

from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=["TV","Radio"],outputCol="new_column_name")
new_fet = feature_assembler.transform(df)
finalized_data = new_fet.select(["new_columns_name","label"])

#-> LinearRegresison

from pyspark.ml.regression import LinearRegression
train_data,test_data =finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol="new_columns_name",labelCol="label")
regressor = regressor.fit(train_data)
regressor.coefficients#a
regreesor.intercept #b
pred_res = regressor.evalutate(test_data)
pred.res.prediction.show()

#pipeline

lr = LinearRegression()
pipeline = Pipeline(stages = [featureIndexer, lr])
model = pipeline.fit(trainingData)
predictions = model.transform(testData)
predictions.select("features", "label", "prediction").show(5)
evaluator = RegressionEvaluator(labelCol = 'label', predictionCol = 'prediction', metricName = 'rmse')
rmse = evaluator.evaluate(predictions)

In [ ]:
from pyspark.sql.functions import udf
quality_udf = udf(lambda x : condition(x), StringType()) #condition bir fonksiyon , istenilien çıktı türü 
#pd.cut veya progress_apply yerin kullanılabilir

In [ ]:
from pyspark.ml.feature import VectorIndexer, StringIndexer, IndexToString
labelIndexer = StringIndexer(inputCol = 'label', outputCol = 'indexedLabel').fit(transformed)
labelIndexer.transform(transformed).show(5, True)

featureIndexer = VectorIndexer(inputCol = 'features', outputCol = 'indexedFeatures', maxCategories = 4).fit(transformed)
featureIndexer.transform(transformed).show(5)
